<a href="https://colab.research.google.com/github/suplab/amazon-bedrock-genai-labs/blob/main/04_Amazon_Bedrock_Flows_and_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Bedrock Flows

Amazon Bedrock Flows is a visual workflow builder that enables developers to create generative AI applications without writing extensive code. This service allows you to orchestrate complex AI workflows by connecting different components through a drag-and-drop interface, making it easier to build and manage AI-powered applications.

The foundation of Bedrock Flows lies in its diverse set of components that handle both flow control and data processing. These components work together to create cohesive, efficient workflows that can scale from simple to complex applications.



## Flow control

Flow control components form the backbone of any Bedrock Flow, managing how data moves through the workflow and determining the logical paths for different scenarios.

### Input and output management

These fundamental components define how data enters and exits your flow, ensuring proper data handling throughout the workflow process.

* **Flow Input Node**

  * Serves as the workflow entry point
  * Defines accepted data types
  * Provides reference points for other nodes

* **Flow Output Node**

  * Extracts data using flow expression language
  * Supports JSON document queries
  * Handles multiple branch outputs

### Flow logic controllers

The logic controllers enable developers to create sophisticated workflows by managing data flow and processing sequences.

* **Condition Node**

  * Implements branching logic
  * Processes multiple named inputs
  * Evaluates conditions sequentially

* **Iterator Node**

  * Processes array items sequentially
  * Provides array item and size outputs
  * Enables parallel processing

* **Collector Node**

  * Aggregates processed items
  * Rebuilds arrays from iterations
  * Combines multiple operation results

## Data processing

Data processing components are the workhorses of Bedrock Flows, handling the actual computation and AI model interactions. These components allow you to interact with foundation models and integrate with various AWS services, forming the core functionality of your generative AI applications.



### AI Integration

The **Prompt Node** is the primary way to interact with foundation models in Bedrock. You can:

* Use inline prompts for quick testing
* Leverage templated prompts from your prompt library
* Configure inference parameters for optimal results

### Service Integration

Bedrock Flows seamlessly connects with other AWS services through specialized nodes:

* S3 for data storage and retrieval
* Lambda for custom business logic
* Lex for natural language processing
* Bedrock Agents and Knowledge Bases for enhanced AI capabilities

# Amazon Bedrock Agents

Amazon Bedrock Agents is a powerful feature that transforms how developers create AI-powered applications capable of completing complex tasks autonomously.

Whether you're building customer service solutions, internal tools, or data analysis applications, Bedrock Agents handles the complex orchestration between foundation models, data sources, and APIs. By combining advanced language models with seamless integration capabilities, it enables your applications to understand user intent, execute actions, and maintain contextual conversations.

Let's explore how Bedrock Agents works, understand its core components, and learn how to leverage its capabilities to build intelligent, task-oriented applications.



## Key components and capabilities

### Foundation Model integration

At the heart of every Bedrock Agent is a foundation model that powers its understanding and decision-making capabilities. When creating an agent, you'll:

* Select from AWS's collection of foundation models available through Amazon Bedrock
* Configure the model to align with your use case through system prompt configuration
* Let the agent use this model to process user inputs, determine when tools are needed, and orchestrate responses

### Agent instructions

Think of agent instructions as your agent's mission statement and operating manual combined. These instructions define the boundaries and personality of your agent, guiding its interactions and decision-making processes.

For example, an HR assistant agent might be instructed to:

* Verify employee eligibility
* Check available leave balance
* Process vacation requests
* Maintain a professional, helpful tone

### Code interpretation
Code interpretation allows agents to generate and execute code in a secure sandbox environment, enabling:

* Real-time data analysis
* Complex calculations
* Format conversions
* Data visualization
* Custom data processing workflows

### Interactive user inputs
The interactive nature of Bedrock Agents is demonstrated through its sophisticated conversation management. During interactions, agents can:

* Request specific information when needed
* Validate user inputs
* Maintain context throughout the interaction
* Guide users through multi-step processes

### Action groups
Action groups serve as your agent's toolkit for executing tasks. These action groups can include Lambda functions, API integrations, and custom tools. Agents can also connect to knowledge bases to access sources like:

* Company policies
* Product documentation
* Technical guides
* FAQs
* Historical data

### Memory
Bedrock Agents can maintain conversation context through its memory capabilities. Memory enables agents to:

* Retain context across multiple user sessions
* Recall and reference past interactions
* Store summarized conversations using the foundation model
* Configure retention periods by:
  * Number of days
  * Number of sessions
* Access relevant historical information when needed

### Knowledge Base Integration
Bedrock Agents can be associated with one or more knowledge bases to enhance their responses. This integration:

* Enables Retrieval Augmented Generation (RAG)
* Allows agents to access domain-specific information such as:
  * Corporate policies
  * Technical documentation
  * Product information
  * Training materials
* Augments LLM responses with verified information
* Provides real-time access to updated company knowledge
* Helps ensure accurate and consistent responses

## Advanced Capabilities

### Orchestration

Bedrock Agents use orchestration prompts to manage complex tasks and interactions. The orchestration process:

* Combines multiple components to build comprehensive responses:
  * Agent instructions
  * Action group definitions
  * Knowledge base content
* Uses system instructions alongside user chat interactions
* Comes with default prompt templates for common scenarios
* Allows customization through advanced prompts for specific needs
* Manages the flow of:
  * User requests
  * Model interactions
  * Function calls
  * Data retrieval

### Mult-agent collaboration

Bedrock Agents can work together as collaborators to handle complex workflows. This collaboration enables:

* Association of multiple specialized agents
* Reuse of existing agent capabilities, such as:
  * Flight booking agents
  * Calendar management agents
  * Data processing agents
* Orchestration of responses across multiple agents
* Division of complex tasks into specialized functions
* Seamless handoff between different agent capabilities
* Maintenance of context across agent interactions

### Strands Agents SDK

Though you can create agents natively in Bedrock, there are frameworks and open-source libraries that make it easier to create agentic applications.
Strands Agents is lightweight and production-ready, supporting many model providers and deployment targets.Key features include:

* **Lightweight and gets out of your way:** A simple agent loop that just works and is fully customizable.
* **Production ready:** Full observability, tracing, and deployment options for running agents at scale.
* **Model, provider, and deployment agnostic:** Strands supports many different models from many different providers.
* **Powerful built-in tools:** Get started quickly with tools for a broad set of capabilities.
* **Multi-agent and autonomous agents:** Apply advanced techniques to your AI systems like agent teams and agents that improve themselves over time.
* **Conversational, non-conversational, streaming, and non-streaming:** Supports all types of agents for various workloads.
* **Safety and security as a priority:** Run agents responsibly while protecting data.

## Creating an HR Assistant Agent using Amazon Bedrock Agents

### Setup and initialize

In [ ]:
!pip install boto3

In [2]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.42.30


In [3]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [7]:
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda', region_name='us-east-1')
bedrock_agent_client = boto3.client('bedrock-agent', region_name='us-east-1')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name='us-east-1')

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

In [ ]:
inference_profile = "us.amazon.nova-micro-v1:0"
foundation_model = inference_profile[3:]
foundation_model

In [ ]:
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_description = "Agent for providing HR assistance to manage vacation time"
agent_instruction = "You are an HR agent, helping employees understand HR policies and manage vacation time"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "Actions for getting the number of available vacations days for an employee and confirm new time off"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

### Creating the AWS Lambda Function

To make our **Amazon Bedrock Agent** more useful, it needs to interact with real data and business logic. In this section, you'll create a backend using AWS Lambda and a lightweight SQLite database to simulate an internal HR system.

This is a common pattern when building AI assistants—offloading specific operations (like database queries, API calls, or business rules) to functions that the model can call as tools. You'll go through the following steps:

✅ Create an employee database using SQLite and populate it with sample HR data

✅ Write a Lambda function that can query and update this data

✅ Provision an IAM role that grants Lambda the permissions it needs

✅ Deploy the Lambda function so it can be used by your Bedrock Agent

By the end of this section, your agent will be able to look up vacation balances and process time-off requests—using real code you wrote and deployed.

First, you need to create the file that will be used to populate the SQLite database. In the next cell, in the Jupyter notebook, run the following code:

In [ ]:
# creating employee database to be used by lambda function
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']

for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y−%m−%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

    # Generate vacation data for the current employee
    for year in range(date.today().year, date.today().year − 3, −1):
        total_vacation_days = random.randint(10, 30)
        days_taken = random.randint(0, total_vacation_days)
        days_available = total_vacation_days − days_taken
        c.execute("INSERT INTO vacations (employee_id, year, employee_total_vacation_days, employee_vacation_days_taken, employee_vacation_days_available) VALUES (?, ?, ?, ?, ?)", (employee_id, year, total_vacation_days, days_taken, days_available))

        # Generate some planned vacations for the current employee and year
        num_planned_vacations = random.randint(0, 3)
        for _ in range(num_planned_vacations):
            start_date = date(year, random.randint(1, 12), random.randint(1, 28)).strftime('%Y−%m−%d')
            end_date = (date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])) + timedelta(days=random.randint(1, 14))).strftime('%Y−%m−%d')
            days_taken = (date(int(end_date[:4]), int(end_date[5:7]), int(end_date[8:])) − date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])))
            c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, days_taken.days))

# Commit the changes and close the connection
conn.commit()
conn.close()

Now, it's time to create the AWS Lambda function. It implements the functionality for get_available_vacations_days for a given employee_id and book_vacations for an employee giving a start and end date.

In [ ]:
%%writefile lambda_function.py
import os
import json
import shutil
import sqlite3
from datetime import datetime


def get_available_vacations_days(employee_id):
    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()

    if employee_id:
        c.execute("""
            SELECT employee_vacation_days_available
            FROM vacations
            WHERE employee_id = ?
            ORDER BY year DESC
            LIMIT 1
        """, (employee_id,))

        available_vacation_days = c.fetchone()

        if available_vacation_days:
            available_vacation_days = available_vacation_days[0]
            conn.close()
            return available_vacation_days
        else:
            conn.close()
            return f"No vacation data found for employed_id {employee_id}"
    else:
        conn.close()
        raise Exception("No employee id provided")

✏️ It defines two core functions `get_available_vacations_days` and `reserve_vacation_time` that interact with an SQLite database. These functions simulate backend business logic that your agent will be able to trigger via tool calls.

The Lambda reads from a local SQLite file stored in `/tmp`, which is the only writable directory in the Lambda execution environment. Before executing the function, it copies the database into `/tmp` if it isn't already present. It uses the event input to determine which function to call and parses parameters accordingly. The response is returned in a format compatible with Amazon Bedrock's tool response structure.

🚨 Make sure that `employee_database.db` is in the same directory as your Lambda deployment package so it can be copied into `/tmp` at runtime. You'll zip up both the lambda_function.py file and the database file in a later step.

Next, you need to create the Lambda IAM role and policy to invoke a Bedrock model.

In [ ]:
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

✅ **Expected output:** Response JSON structure.

```json
{
  'ResponseMetadata':
  {
    'RequestId': 'a5645679-43a2-4564-abfc-7ebc46ecba56',
    'HTTPStatusCode': 200,
    'HTTPHeaders':
    {
      'date': 'Mon, 09 Jun 2025 20:25:10 GMT',
      'x-amzn-requestid': 'a5645679-43a2-4564-abfc-7ebc46ecba56',
      'content-type': 'text/xml',
      'content-length': '212'
    },
    'RetryAttempts': 0
  }
}
```

In [ ]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.write("employee_database.db")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

### Create Amazon Bedrock Agent

**Time to Create the Agent!**

Before you can launch the HR agent into action, you need to give it the right permissions. In this step, you'll:

🔐 Create the agent policies that allow it to invoke Bedrock foundation models.  
🛡️ Create the IAM role for the agent and attach the policies.

These steps will allow the agent to safely and successfully use the tools you define.

First, you will create the IAM policy that the agent will need to use the inference profile and invoke the model.


In [ ]:
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:*::foundation-model/{foundation_model}",
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        },
        {
            "Sid": "AmazonBedrockAgentBedrockGetInferenceProfile",
            "Effect": "Allow",
            "Action":  [
                "bedrock:GetInferenceProfile",
                "bedrock:ListInferenceProfiles",
                "bedrock:UseInferenceProfile"
            ],
            "Resource": [
                f"arn:aws:bedrock:*:*:inference-profile/{inference_profile}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)

Now, you need to create the IAM role that the agent will need to assume to perform its duties. You will also attach an IAM policy to the role.

In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Principal": {
            "Service": "bedrock.amazonaws.com"
        },
        "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

✅ **Expected output:** Response JSON structure.

```json
{'ResponseMetadata': {'RequestId': 'a5645679-43a2-4564-abfc-7ebc46ecba56',
'HTTPStatusCode': 200,
'HTTPHeaders': {'date': 'Mon, 09 Jun 2025 20:25:10 GMT',
'x-amzn-requestid': 'a5645679-43a2-4564-abfc-7ebc46ecba56',
'content-type': 'text/xml',
'content-length': '212'},
'RetryAttempts': 0}}
```

Once the needed IAM role is created, you can use the Bedrock Agent client to create a new agent. To do so, you use the `create_agent` API. It requires an agent name, foundation model ID and instructions. You can also provide an agent description.

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=inference_profile,
    instruction=agent_instruction,
)
agent_id = response['agent']['agentId']
agent_id, response

✅ **Expected output:** Agent ID and response JSON structure.

```
('T469IP3A2B',
{'ResponseMetadata': {'RequestId': '71c580d7-b835-4fec-bf43-2aed4eeb282f',
'HTTPStatusCode': 202,
'HTTPHeaders': {'date': 'Mon, 09 Jun 2025 20:29:24 GMT',
    'content-type': 'application/json',
    'content-length': '692',
    'connection': 'keep-alive',
    'x-amzn-requestid': '71c580d7-b835-4fec-bf43-2aed4eeb282f',
    'x-amz-apigw-id': 'L6eVtEjFoAMEicg=',
    'x-amzn-trace-id': 'Root=1-68474424-5063abe010f0ef345b1f5cda'},
'RetryAttempts': 0},
'agent': {'agentArn': 'arn:aws:bedrock:<region>:<account>:agent/T469IP3A2B',
'agentCollaboration': 'DISABLED',
'agentId': 'T469IP3A2B',
'agentName': 'hr-assistant-function-def',
'agentResourceRoleArn': 'arn:aws:iam::<account>:role/AmazonBedrockExecutionRoleForAgents_hr-assistant-function-def',
'agentStatus': 'CREATING',
'createdAt': datetime.datetime(2025, 6, 9, 20, 29, 24, 436469, tzinfo=tzutc()),
'description': 'Agent for providing HR assistance to manage vacation time',
'foundationModel': 'us.amazon.nova-micro-v1:0',
'idleSessionTTLInSeconds': 1800,
'instruction': 'You are an HR agent, helping employees understand HR policies and manage vacation time',
'orchestrationType': 'DEFAULT',
'updatedAt': datetime.datetime(2025, 6, 9, 20, 29, 24, 436469, tzinfo=tzutc())}})
```

✏️ Note that the agent created is not yet prepared. Later, you will prepare and use the agent.

🎉 The agent has now been created, but it doesn't have any tools yet. In the next step, you'll create an action group to associate the tool to the agent.

### Create Agent Action Group

You will now create an agent action group that uses the Lambda function created earlier. The
`create_agent_action_group` function provides this functionality. You will use DRAFT as the agent version since you haven't yet created an agent version or alias. To inform the agent about the action group capabilities, you provide an action group description.

In this example, you provide the Action Group functionality using a **functionSchema**. You can alternatively provide an APISchema.

To define the functions using a function schema, you need to provide the name, description and parameters for each function.

You will first create the agent function schema. This is used to define the name, description, and parameters for the different actions available to the agent.

In [ ]:

agent_functions = [
    {
        'name': 'get_available_vacations_days',
        'description': 'get the number of vacations available for a certain employee',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee to get the available vacations",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee for which time off will be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "the start date for the vacation time",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "the end date for the vacation time",
                "required": True,
                "type": "string"
            }
        }
    },
]

To create the action group, you will use the create_agent_action_group API.

In [ ]:
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

agent_action_group_response

✅ **Expected output:** Response JSON structure.

```json
{'ResponseMetadata': {'RequestId': '68759a87-f4c4-48ef-a749-dc4479da5eac',
'HTTPStatusCode': 200,
'HTTPHeaders': {'date': 'Mon, 09 Jun 2025 20:31:28 GMT',
'content-type': 'application/json',
'content-length': '1334',
'connection': 'keep-alive',
'x-amzn-requestid': '68759a87-f4c4-48ef-a749-dc4479da5eac',
'x-amz-apigw-id': 'L6epHGGCIAMEMAg=',
'x-amzn-trace-id': 'Root=1-684744a0-3c9a129513498d7b466ba439'},
'RetryAttempts': 0},
'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:<region>:<account>:function:hr-assistant-function-def-<region>-<account>'},
'actionGroupId': 'HSCFRJRSN2',
'actionGroupName': 'VacationsActionGroup',
'actionGroupState': 'ENABLED',
'agentId': 'T469IP3A2B',
'agentVersion': 'DRAFT',
'createdAt': datetime.datetime(2025, 6, 9, 20, 31, 28, 433727, tzinfo=tzutc()),
'description': 'Actions for getting the number of available vaction days for an employee and confirm new time off',
'functionSchema': {'functions': [{'description': 'get the number of vacation days available for a certain employee',
'name': 'get_available_vacations_days',
'parameters': {'employee_id': {'description': 'the id of the employee to get the available vacation days',
'required': True,
'type': 'integer'}},
'requireConfirmation': 'DISABLED'},
{'description': 'reserve vacation time for a specific employee - you need all parameters to reserve vacation time',
'name': 'reserve_vacation_time',
'parameters': {'end_date': {'description': 'the end date for the vacation time',
'required': True,
'type': 'string'},
'employee_id': {'description': 'the id of the employee for which time off will be reserved',
'required': True,
'type': 'integer'},
'start_date': {'description': 'the start date for the vacation time',
'required': True,
'type': 'string'}},
'requireConfirmation': 'DISABLED'}]},
'updatedAt': datetime.datetime(2025, 6, 9, 20, 31, 28, 433727, tzinfo=tzutc())}}
```

At this point, the action group has been created, but the AWS Lambda function resource policy needs to be modified to allow the agent to invoke the function associated with the action group.

Modify the resource policy on the AWS Lambda function to allow the specific bedrock agent to invoke it.

In [ ]:
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

🎉 The action group has been created and the agent has the needed permissions to invoke the AWS Lambda function. In the next step, you will prepare the agent and create an agent alias.

### Prepare the Agent and Create an Agent Alias

The prepare_agent API creates a DRAFT version of the agent that can be used for internal testing.

To prepare the agent, in the next cell, copy and paste the following code. Then, run the code:

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
response

✅ **Expected output:** Response JSON structure.

```json
{'ResponseMetadata': {'RequestId': 'd92ea1b1-82d2-4abd-b500-59f9c00eaf35',
'HTTPStatusCode': 202,
'HTTPHeaders': {'date': 'Mon, 09 Jun 2025 20:33:46 GMT',
'content-type': 'application/json',
'content-length': '119',
'connection': 'keep-alive',
'x-amzn-requestid': 'd92ea1b1-82d2-4abd-b500-59f9c00eaf35',
'x-amz-apigw-id': 'L6e-oFlUIAMELjA=',
'x-amzn-trace-id': 'Root=1-6847452a-4455e8d203f86e532142249f'},
'RetryAttempts': 0},
'agentId': 'T469IP3A2B',
'agentStatus': 'PREPARING',
'agentVersion': 'DRAFT',
'preparedAt': datetime.datetime(2025, 6, 9, 20, 33, 46, 488240, tzinfo=tzutc())}
```

✏️ Locate and take note of the **agentId** in the response output, as you will need it in the next step.

In [ ]:
# Replace <FILL_ME_IN> with the agent ID from the output from the last cell.
response = bedrock_agent_client.create_agent_alias(
    agentAliasName='test-alias-1',
    agentId="<FILL_ME_IN>"
)

response

✅ **Expected output:** Response JSON structure.

```json
{'ResponseMetadata': {'RequestId': 'fe9cc44f-c94e-49fb-bc04-b3d071f65fa0',
'HTTPStatusCode': 202,
'HTTPHeaders': {'date': 'Mon, 09 Jun 2025 20:35:26 GMT',
'content-type': 'application/json',
'content-length': '382',
'connection': 'keep-alive',
'x-amzn-requestid': 'fe9cc44f-c94e-49fb-bc04-b3d071f65fa0',
'x-amz-apigw-id': 'L6fOPHgtoAMEicg=',
'x-amzn-trace-id': 'Root=1-6847458d-5c8ebd9171af4d4e311f2d46'},
'RetryAttempts': 0},
'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:<region>:<account>:agent-alias/T469IP3A2B/UMI7BW4KDX',
'agentAliasId': 'UMI7BW4KDX',
'agentAliasName': 'test-alias-1',
'agentAliasStatus': 'CREATING',
'agentId': 'T469IP3A2B',
'aliasInvocationState': 'ACCEPT_INVOCATIONS',
'createdAt': datetime.datetime(2025, 6, 9, 20, 35, 26, 83771, tzinfo=tzutc()),
'routingConfiguration': [{}],
'updatedAt': datetime.datetime(2025, 6, 9, 20, 35, 26, 83771, tzinfo=tzutc())}}
```

Locate and take note of the **agentAliasId** in the response output, as you will need it in the next step. Ensure you are copying the **agentAliasId** and not the agentId. These are two different values, and if you mix them up, the exercise will not work.

In the next cell, run the following command: **agent_alias_id = "<FILL_ME_IN>"**

Replace <FILL_ME_IN> with the agent alias ID. Ensure both of the < > characters are removed and only the ID in double quotes is present. Then, run the code.

🎉 The agent has been prepared and an alias has been deployed. Next up, you will test the agent!

### Invoke the Agent
Now that you've created the agent, let's use the bedrock-agent-runtime client and the invoke_agent API to invoke this agent and perform some tasks.

In [ ]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How much vacation does the employee with employee_id set to 1 have available?",
    agentId=agent_id,
    agentAliasId=agent_alias_id,
    sessionId=session_id,
    enableTrace=enable_trace,
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

✏️ This will invoke the agent with the prompt: **How much vacation does the employee with employee_id set to 1 have available?**

The agent will think through the prompt, and choose to invoke the tool to look up the employee vacation balance. This will invoke the AWS Lambda function which will read the data from the SQLite database. The agent will receive the response from the function and work it into its final response.

✅ **Expected output:** Response structure and agent answer.

```
Final answer ->
The employee with employee_id 1 has 14 vacation days available.
```

Now try a follow up prompt to reserve one day off for the employee.

In [ ]:
# Replace <FILL_ME_IN> with a future date in YYYY-MM-DD format.
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="Great. please reserve one day of time off for the employee with employee_id set to 1 for <FILL_ME_IN>",
    agentId=agent_id,
    agentAliasId=agent_alias_id,
    sessionId=session_id,
    enableTrace=enable_trace,
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

✅ **Expected output:** Agent confirms the vacation was reserved.

```
Final answer ->
The vacation has been successfully reserved for the employee with employee_id 1 for 2025-06-01.
```

Try another prompt to check to see if the agent was able to reserve time off for the employee. The amount of available days should have decreased by one.

In [ ]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="How much vacation does the employee with employee_id set to 1 have available?",
    agentId=agent_id,
    agentAliasId=agent_alias_id,
    sessionId=session_id,
    enableTrace=enable_trace,
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

event_stream = agentResponse['completion']
try:
    for event in event_stream:
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

✅ **Expected output:** Agent reflects updated vacation days.

```
Final answer ->
The employee with employee_id 1 has 13 vacation days available after reserving one day.
```

### Cleanup
The next steps demonstrate how to delete the agent and associated resources.

To delete the agent, you need to:

❌ Update the action group to disable it

❌ Delete agent action group

❌ Delete agent

❌ Delete lambda function

❌ Delete the created IAM roles and policies


In [ ]:
# Step 1: Update and delete the action group.
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

# Step 2: Delete the agent alias.
response = bedrock_agent_client.delete_agent_alias(
    agentAliasId=agent_alias_id,
    agentId=agent_id
)

# Step 3: Delete the agent.
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

# Step 4: Delete the Lambda function.
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

# Step 5: Delete the IAM roles and policies.
for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')

iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)